# Causal discovery and effect estimation case study



In [3]:
%pip install statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 2.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     

# first install tigramite developer branch from https://github.com/jakobrunge/tigramite.git
import tigramite
from tigramite import data_processing as pp
from tigramite.toymodels import structural_causal_processes
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
#from tigramite.independence_tests import ParCorr, GPDC, CMIknn, RobustParCorr
from tigramite.independence_tests.oracle_conditional_independence import OracleCI
from tigramite.independence_tests.parcorr import ParCorr
from tigramite.independence_tests.gpdc import GPDC
from tigramite.independence_tests.cmiknn import CMIknn
from tigramite.independence_tests.robust_parcorr import RobustParCorr

from tigramite.causal_effects import CausalEffects
from tigramite.lpcmci import LPCMCI

import statsmodels
# Seaborn for nice scatter plots
import seaborn as sns
import pandas as pd

In [5]:
np.random.seed(42)

# Load dataset
which = 'synthetic' #'climate'

if which == 'synthetic':
    data = np.load("dataset_no1.npy")
    var_names = [r'$X^0$', r'$X^1$', r'$X^2$', r'$X^3$']
else:
    data = np.loadtxt("WPAC_CPAC_EPAC_ATL.txt")
    var_names = ["WPAC", "CPAC", "EPAC", "ATL"]


# data = model_data(1000)

# Specify variable names

# Initialize tigramite dataframe object
dataframe = pp.DataFrame(data, var_names=var_names)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_no1.npy'

## Pre-analysis steps

- Check stationarity
- Check marginal and joint distributions
- Check lagged dependencies


First, we plot the time series. This can be done with the function ``tp.plot_timeseries``

In [ ]:
tp.plot_timeseries(dataframe); plt.show()

It looks stationary (how to test this?) and doesn't contain missing values. 

## Nonlinearity and noise distributions

Here we use kernel dendity plots to get an impression of the type of dependencies (linear, nonlinear) and noise distributions (for example, Gaussian).

In [ ]:
sns.set(style="ticks")
df = pd.DataFrame(data)
sns.pairplot(df, diag_kind = 'kde'); plt.show()

Variable 1 seems to be non-Gaussian, the other's have more Gaussian marginals. From the density plots there is no obvious nonlinearity. There is much more one can test, but let's continue with causal methods.

To help choose a causal discovery method for time series, we need to know the maximal time delay of direct causal relations.


## Time delays

We assess  whether time delays are present in the time series by looking at the lagged cross correlation function (because the joint density plots did not indicate nonlinear dependencies here, otherwise use nonlinear method). This can help to identify which maximal time lag ``tau_max`` to choose in the causal algorithm.

In [ ]:
ci_test = ParCorr() #(significance='analytic')
# ci_test = RobustParCorr() #(significance='analytic')

pcmci = PCMCI(
    dataframe=dataframe, 
    cond_ind_test=ci_test,
    verbosity=0)

In [ ]:
correlations = pcmci.get_lagged_dependencies(tau_max=10, val_only=True)['val_matrix']

In [ ]:
lag_func_matrix = tp.plot_lagfuncs(val_matrix=correlations, setup_args={'var_names':var_names, 
                                    'x_base':5, 'y_base':.5})

Since the dependencies decay beyond a maximum lag of around 3, we can choose ``tau_max=3`` for causal inference methods. 

## Causal discovery

- Granger causality
- PCMCI causal discovery algorithm
- PCMCI+ causal discovery algorithm
- LiNGAM causal discovery method

In [ ]:
def granger_causality(data, i, j, maxlags=5, alpha_lev=0.05):
    """Granger causality test using statsmodels."""
    
    import statsmodels.tsa.api as tsa

    tsamodel = tsa.var.var_model.VAR(data)
    results = tsamodel.fit(maxlags=maxlags)

    return results.test_causality(j, causing=i).pvalue < alpha_lev

In [ ]:
alpha_lev = 0.01
for i in range(data.shape[1]):
    for j in range(i+1, data.shape[1]):
        # Granger
        granger_result = granger_causality(data[:1000], i=i, j=j, maxlags=5, alpha_lev=alpha_lev)
        if granger_result == 1:
            print("Granger causality %s --> %s" % (var_names[i], var_names[j]))

### PCMCI

Recall that PCMCI assumes lagged dependencies.

In [ ]:
# Initialization
# parcorr = ParCorr(significance='analytic')
# pcmci = PCMCI(
#     dataframe=dataframe, 
#     cond_ind_test=parcorr,
#     verbosity=0)


In [ ]:
# Significance level, here more a hyper-parameter than a statistical parameter
alpha_level = 0.01
tau_max = 2
results = pcmci.run_pcmci(tau_max=tau_max, tau_min=0, alpha_level=alpha_level)
tp.plot_time_series_graph(graph=results['graph'], save_name=None, 
            var_names=var_names, 
                          figsize=(6, 5), 
                        )
tp.plot_graph(graph=results['graph'], save_name=None, 
            var_names=var_names, 
                          figsize=(6, 5), 
                        ); plt.show()

### PCMCI+ algorithm

Now we use the PCMCI+ algorithm that can also cope with contemporaneous links. We choose partial correlation, thus limiting ourselves to linear dependencies.

In [ ]:
alpha_lev = 0.01
tau_max = 2
results = pcmci.run_pcmciplus(pc_alpha=alpha_lev, 
                    tau_min=0, tau_max=tau_max)

In [ ]:
tp.plot_time_series_graph(graph=results['graph'], save_name=None, 
            var_names=var_names, 
                          figsize=(6, 5), 
                        )
tp.plot_graph(graph=results['graph'], save_name=None, 
            var_names=var_names, 
                          figsize=(6, 5), 
                        ); plt.show()

PCMCI+ finds links 1-->2-->3 and an undirected link 0--1 due to Markov equivalence. 

In [ ]:
lpcmci = LPCMCI(
    dataframe=dataframe, 
    cond_ind_test=ci_test,
    verbosity=0)
latent_results = lpcmci.run_lpcmci(tau_max=tau_max, pc_alpha=alpha_level)

In [ ]:
tp.plot_time_series_graph(graph=latent_results['graph'], save_name=None, 
            var_names=var_names, 
                          figsize=(6, 5), 
                        )
tp.plot_graph(graph=latent_results['graph'], save_name=None, 
            var_names=var_names, 
                          figsize=(6, 5), 
                        ); plt.show()

### LiNGAM

We now try to estimate the causal direction for the 0--1 link using a restricted structural causal model, here a LiNGAM assuming non-Gaussianity (of at least one variable) and linearity.

In [ ]:
def lingam(data, i, j):
    """Performs bivariate LiNGAM causality test.

    The bivariate LiNGAM model assumes linear dependencies and that
    either X or eta^Y is non-Gaussian. Here we also assume
    no common drivers and that i and j are dependent which needs to
    be tested with a correlation test beforehand. The independence
    test is done with distance correlation (tigramite package).   
    
    """

    def indep_test(one, two):
        ind_test = GPDC()

        array = np.vstack((one, two))
        xyz = np.array([0,1])

        dim, n = array.shape
        value = ind_test.get_dependence_measure(array, xyz)
        pval = ind_test.get_analytic_significance(value, T=n, dim=dim, xyz=xyz)

        return pval

    x = data[:, i].reshape(-1, 1)
    y = data[:, j].reshape(-1, 1)

    # Test causal model x --> y
    beta_hat_y = np.linalg.lstsq(x, y, rcond=None)[0]
    yresid = y - np.dot(x, beta_hat_y)
    pval_xy = indep_test(x.flatten(), yresid.flatten())

    # Test causal model y --> x
    beta_hat_x = np.linalg.lstsq(y, x, rcond=None)[0]
    xresid = x - np.dot(y, beta_hat_x)
    pval_yx = indep_test(y.flatten(), xresid.flatten())

    if pval_xy >= pval_yx:
        return 1
    else:
        return -1

    return pval, value

In [ ]:
graph_effects = results['graph'].copy()

# LiNGAM on contemporaneous pairs
for i in range(data.shape[1]):
    for j in range(i+1, data.shape[1]):
        # LiNGAM
        if results['graph'][i, j, 0] == 'o-o' or results['graph'][i, j, 0] == 'x-x':
            lingam_result = lingam(data[:1000], i, j)
            if lingam_result == 1:
                print("LiNGAM on contemp pair %s --> %s" % (var_names[i], var_names[j]))
                graph_effects[i,j,0] = '-->'
                graph_effects[j,i,0] = '<--'
            else:
                print("LiNGAM on contemp pair %s --> %s" % (var_names[j], var_names[i]))
                graph_effects[i,j,0] = '<--'
                graph_effects[j,i,0] = '-->'

FINAL GRAPH of causal discovery exercise:

In [ ]:
# graph_effects = results['graph'].copy()
# Orient link 1 --> 0 based on LiNGAM result.

tp.plot_time_series_graph(graph=graph_effects,
                        var_names=var_names, 
                          figsize=(6, 5), 
#                           special_nodes={(2, 0):'red', (3, 0):"blue"}
                        )
tp.plot_graph(graph=graph_effects, save_name=None, 
            var_names=var_names, 
                          figsize=(6, 5), 
                        ); plt.show()


## Estimating causal effects

First, we need a function to give us the regression coefficients.

In [ ]:
def causal_effect(X, Y, Z=None):
    """Yields the regression coefficient beta_x in the model
    
    Y = beta_x * X + beta_z * Z + noise
    
    Also an intercept is fitted.
    """
    
    n_obs = len(Y)
    
    if Z is None:
        return np.linalg.lstsq(np.hstack((X.reshape(len(X), 1), 
                                          np.ones(n_obs).reshape(-1,1))), 
                                Y.reshape(-1, 1), 
                                rcond=None)[0][0]
    else:
        if Z.ndim == 1:
            Z = Z.reshape(len(Z), 1)
        return np.linalg.lstsq(np.hstack((X.reshape(len(X), 1), 
                                          Z, 
                                          np.ones(n_obs).reshape(-1,1))), 
                               Y.reshape(-1, 1), 
                               rcond=None)[0][0]

def causal_effect_incl_error(X, Y, Z=None, n_boots=1000):
    """Bootstrap standard error estimation."""
    
    n_obs = len(Y)

    estimate_boot = np.zeros(n_boots)
    for b in range(n_boots):
        rand = np.random.randint(0, n_obs, n_obs)
        if Z is None:
            estimate_boot[b] = causal_effect(X[rand], Y[rand], Z=None)      
        else:
            estimate_boot[b] = causal_effect(X[rand], Y[rand], Z=Z[rand])
        
    return causal_effect(X, Y, Z=Z), estimate_boot.std()

Now let's look at different causal effects among the (lagged) variables. We will use linear regression and can use the learned causal identification methods to identify which conditions (covariates) to use by reading them off the time series graph. 

In [ ]:
tp.plot_time_series_graph(graph=graph_effects,
                        var_names=var_names, 
                          figsize=(6, 5), 
#                           special_nodes={(2, 0):'red', (3, 0):"blue"}
                        ); plt.show()

In [ ]:
print("Causal effect = %.2f +/- %.2f" % causal_effect_incl_error(X=data[1:,2], 
                                                                 Y=data[1:,3], 
                                                                 Z=data[:-1,[1,2]]))

In [ ]:
print("Causal effect = %.2f +/- %.2f" % causal_effect_incl_error(X=data[1:,2], 
                                                                 Y=data[1:,3], 
                                                                 Z=data[:-1,[3]]))

## Advanced task: Nonlinear causal effect estimation

Below code for you to play around!

In [ ]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

In [ ]:
def estimate_causal_effect(X, Y, adjustment_set=None, model=None):
    causal_effects = CausalEffects(graph=graph_effects, 
                               graph_type='stationary_dag', 
                               X=X, Y=Y,  
                               verbosity=0)
    
    if model is None: model = LinearRegression()
    # Fit causal effect model from observational data
    causal_effects.fit_total_effect(
            dataframe=dataframe, 
            estimator=model,
            adjustment_set=adjustment_set,
            )

    # Set X to intervened values
    intervention_data = np.ones((1, 1))
    y1 = causal_effects.predict_total_effect( 
            intervention_data=intervention_data, 
            )
    intervention_data = np.zeros((1, 1))
    y2 = causal_effects.predict_total_effect( 
            intervention_data=intervention_data, 
            )

    for y in Y:
        beta = (y1 - y2)
        print("Causal effect = %.2f" %(beta))

In [ ]:
estimate_causal_effect(X = [(1, -1)], Y = [(3,0)], adjustment_set=[])
estimate_causal_effect(X = [(1, -1)], Y = [(3,0)], adjustment_set=[(3, -1), (2, -1)])

estimate_causal_effect(X = [(2, 0)], Y = [(3,0)], adjustment_set=[(1, -1), (2, -1)])
estimate_causal_effect(X = [(2, 0)], Y = [(3,0)], adjustment_set=[(3, -1),])

In [ ]:
def model_data(n_obs):
    # 0, 1 are uniform, while 2, 3 are Gaussian
    X = np.random.randn(n_obs, 4)
    X[:, 0] = 2.*np.random.rand(n_obs) - 1
    X[:, 1] = 2.*np.random.rand(n_obs) - 1
    
    for t in range(2, n_obs):
        # X[t,1] += 0.9*X[t-1,1
        X[t, 0] += .5 * X[t, 1]   #+ 0. * X[t - 1, 0]

        X[t, 2] += .75 * X[t - 1, 2] + 0.8 * X[t - 1, 1]
        X[t, 3] += .75 * X[t - 1, 3] + 0.8 * X[t, 2]

    return X